# Admissions Classifier

Binary classifier that predicts **Accepted** vs **Rejected** for applications using GPA, SAT score, and extracurricular activities.

**Objective**: Train a neural network on the [Kaggle student admission dataset](https://www.kaggle.com/datasets/amanace/student-admission-dataset).

**Model type**: Binary classification (sigmoid output, binary cross-entropy).

**Workflow**:
1. Load data
2. Preprocess and binarize target (Accepted=1, Rejected/Waitlisted=0)
3. Train/test split, scale features
4. Build and train neural network
5. Evaluate and save model + scaler

## Run in the browser (no local setup)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adzuci/admissions-classifier/blob/main/admissions_classifier.ipynb)

> In Colab: upload `student_admission_dataset.csv` when prompted (download from Kaggle).

In [ ]:
try:
    import google.colab
    get_ipython().run_line_magic('pip', 'install -q tensorflow pandas')
except (ImportError, NameError):
    pass  # local: deps assumed installed

## STEP 1: Imports and SimpleScaler

In [ ]:
import pickle
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers


class SimpleScaler:
    """Minimal scaler with fit/transform (no sklearn)."""

    def fit(self, X):
        self.mean_ = np.mean(X, axis=0)
        self.scale_ = np.std(X, axis=0)
        self.scale_[self.scale_ == 0] = 1.0
        return self

    def transform(self, X):
        return (X - self.mean_) / self.scale_


print(f"TensorFlow: {keras.__version__}")

## STEP 2: Load data

In [ ]:
# In Colab: upload the CSV when prompted (download from Kaggle)
try:
    from google.colab import files
    uploaded = files.upload()
    csv_name = list(uploaded.keys())[0]
except ImportError:
    csv_name = "student_admission_dataset.csv"

df = pd.read_csv(csv_name)
df.head(10)

## STEP 3: Preprocess

In [ ]:
# Binarize: Accepted=1, Rejected/Waitlisted=0
df["Admit"] = (df["Admission_Status"] == "Accepted").astype(int)

feature_cols = ["GPA", "SAT_Score", "Extracurricular_Activities"]
X = df[feature_cols].values.astype(np.float32)
y = df["Admit"].values

print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"Class balance: Admit=1 {y.sum()}, Admit=0 {len(y) - y.sum()}")

In [ ]:
# Train/test split (80/20)
np.random.seed(42)
idx = np.random.permutation(len(X))
split = int(0.8 * len(X))
train_idx, test_idx = idx[:split], idx[split:]

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

# Scale features
scaler = SimpleScaler()
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

## STEP 4: Build and train model

In [ ]:
model = keras.Sequential([
    keras.Input(shape=(3,)),
    layers.Dense(8, activation="relu"),
    layers.Dense(1, activation="sigmoid"),
])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
history = model.fit(X_train_s, y_train, epochs=50, batch_size=32, verbose=1)

## STEP 5: Evaluate

In [ ]:
loss, acc = model.evaluate(X_test_s, y_test, verbose=0)
print(f"Test accuracy: {acc:.4f}")

preds = (model.predict(X_test_s, verbose=0) > 0.5).astype(int).flatten()
print(f"Sample predictions: {preds[:15]}...")

## STEP 6: Save model and scaler

In [ ]:
model.save("model.keras")
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)
print("Saved model.keras and scaler.pkl")
print("In Colab: use Files pane to download these files.")